# Cargar datos y librerias

In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import shift
from sklearn.linear_model import LinearRegression

In [2]:
from warnings import filterwarnings
filterwarnings("ignore")

In [3]:
import chart_studio.plotly as ply
import cufflinks as cf
cf.go_offline()

In [4]:
def cargar():
    # variables climaticas
    consumofeb=pd.read_excel(r'./Consumo_feb.xlsx').set_index("fecha").loc["2007-01-01":"2020-03-30"] #desde 2007 hasta 31 de marzo de 2020
    consumofeb = consumofeb.drop(["prom", 'Lluvia', 'Velocidad_viento'], axis = 1)
    #variables de consumo MWh
    consumo=pd.read_excel('./Data1.xlsx').set_index("fecha").loc["2007-01-01":"2020-03-30"] 
    return consumo, consumofeb

consumo, consumofeb = cargar()

# pred clima

In [5]:
def kronecker(data1:'Dataframe 1',data2:'Dataframe 2'):
    Combinacion = pd.DataFrame(); d1 = pd.DataFrame()
    for i in data2.columns:
        d1 = data1.multiply(data2[f"{i}"] , axis="index")
        d1.columns = [f'{i}_{j}' for j in data1.columns]
        Combinacion = pd.concat([Combinacion, d1], axis = 1)
    return Combinacion

In [16]:
def pba(j, n):
    # armonicos
    f, asd = signal.periodogram(consumofeb[j], 1)

    picos = pd.DataFrame(asd, 1/(f), columns=["potencia"])
    picos = picos.sort_values(by="potencia",ascending=False).reset_index().head(12)
    picos.columns = ["periodo", "potencia"]
    #fourier
    sencos = pd.DataFrame(index = consumofeb.index)
    t = np.arange(1,len(consumofeb)+1)
    sencos["t"]=t
    for i  in  picos.periodo:
            sencos[f"{i:.2f}_sen"] = np.abs(np.sin(((2*np.pi)/(i))*t))
            sencos[f"{i:.2f}_cos"] = np.abs(np.cos(((2*np.pi)/(i))*t))
    sencos['ones']=1
    sencos['sen1']=np.abs(np.sin(((2*np.pi)/(365.25))*t))
    sencos['cos1']=np.abs(np.cos(((2*np.pi)/(365.25))*t))
    
    sencos=kronecker(sencos, triangular())
    #regresion
    X_train, X_test, y_train, y_test = train_test_split(sencos, consumofeb[j], test_size=n, shuffle = False)
    
    reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
    y_predict=reg.predict(X_test)
    
    nov=pd.DataFrame(np.concatenate([reg.predict(X_train),y_predict]),index=consumofeb.index,columns=[f"{j}"])
    return nov

def modelosclima(n):
    ca = pd.DataFrame(index = consumo.index)
    for i in ["mín", "max", "Nubosidad"]:
        ca = pd.concat([ca, pba(i, n)], axis = 1)
    return ca

modclima = modelosclima(8)

# funciones

In [17]:
def difs():
    # Inicializar posicion de datos variables difusas

    inv1 = [310.5-365.25, 8.6529, 138.941176470588]
    pr = [8.6529, 138.941176470588, 221.9375]
    vr = [138.941176470588, 221.9375, 310.5]
    ot = [221.9375, 310.5, 365.25+8.6529]
    ot2 = [221.9375-365.25, 310.5-365.25, 8.6529]
    inv2 = [310.5, 365.25+8.6529, 365.25+138.941176470588]

    height = {'Invierno1': inv1,'Primavera': pr,'verano': vr,'Otoño': ot,'Otoño2': ot2,'Invierno2': inv2,} 
    return height

# para crear variables difusas
def triangular():
    
    # Se crea x
    ma = pd.DataFrame();
    x = consumofeb.index.dayofyear.to_list()
    # Se crean variables a, b y c
    height = difs()
    for hei in height:
        ba = pd.DataFrame(x, columns=["x"])
        ba["a"] = height[hei][0]
        ba["b"] = height[hei][1]
        ba["c"] = height[hei][2]
        # Variable difusa
        ca = pd.DataFrame()
        ca["bo"] = (ba.x-ba.a)/(ba.b-ba.a)
        ca["ba"] = (ba.c-ba.x)/(ba.c-ba.b)
        ca = ca.min(axis = 1)
        ca[ca<0]=0
        ma = pd.concat([ma, ca], axis = 1) # Se concatenan

    ma.index = consumofeb.index
    ma.columns = height.keys()
    return ma

In [18]:
def festivos():
    festivos=pd.read_excel('./Festivos.xlsx')
    festivos2=pd.read_excel('./Festivos2.xlsx')

    Dum = pd.DataFrame(index = consumo.index)

    for col in festivos.columns:
        Dum[f"{col}"] = consumo.index.isin(festivos[f"{col}"])
        Dum[col] = Dum[col].replace([False, True],[0, 1])
        clean = Dum[col].values
        Dum[col] = Dum[col]+shift(clean, 1, cval=0)*.4
        Dum[col] = Dum[col]+shift(clean, -1, cval=0)*.4
        Dum[col] = Dum[col]+shift(clean, 2, cval=0)*.1
        Dum[col] = Dum[col]+shift(clean, -2, cval=0)*.1
        Dum[col] = Dum[col]/2

#     Dum["ones"] = 1
    tm = np.arange(1,len(consumo.index)+1)  
    Dum["t"] = tm/np.max(tm)
    
    for col in festivos2.columns:
        Dum[col] = consumo.index.isin(festivos2[col])
    Dum = Dum.replace([False, True],[0, 1])
    
    return Dum

In [19]:
def dummies(n):   
    # variables eureka y clima
    consumo2 = pd.DataFrame(index=consumo.index)
    consumo2["num"] = np.arange(1,len(consumo.index)+1)
    consumo2["day"] = consumo.index.weekday
    consumo2["eureka1"] = consumo2.day**3
    consumo2["eureka2"] = modclima["max"]*consumo2.num
    consumo2["month"] = consumo.index.month
    X = consumo2.join(modclima)
    # normalizacion eureka-clima
    X_norm = X/X.max()
    
    # variables gplearn
    X_norm["gplearn1"]=np.cos(X_norm.day)*X_norm.num
    X_norm["gplearn2"]=X_norm.num*X_norm["mín"]
    X_norm["gplearn3"]=3.319**X_norm["max"]*np.cos(X_norm.day)
    X_norm["gplearn8"]=3.319**X_norm["max"]*X_norm.num 
    X_norm["gplearn4"]=np.cos(X_norm.day)*np.sin(X_norm.day)
    X_norm["gplearn6"]=X_norm["max"]**2*X_norm["mín"]**2
    X_norm["gplearn7"]=1/np.cos(X_norm.day)
    X_norm["gplearn10"]=X_norm["max"]*X_norm["mín"]**3*np.cos(X_norm.month)**2
    
    X_norm=X_norm.drop(['mín', 'max', 'Nubosidad'],axis=1)
    # variables dummies
    X_norm = pd.get_dummies(X_norm, columns=["month", "day"], prefix=["month", "day"], drop_first=True)

    # variables festivos y difusa de clima
    X_norm = X_norm.join(festivos())
    X_norm = X_norm.join(triangular())
    
    # variables entre y fin
    X_norm["fin"]=consumo.index.weekday
    X_norm["entre"]=consumo.index.weekday
    X_norm["fin"]=X_norm["fin"].replace([0,1,2,3,4,5,6],[.5,0,0,0,.5,1,1])
    X_norm["entre"]=X_norm["entre"].replace([0,1,2,3,4,5,6],[.5,1,1,1,.5,0,0])
    return X_norm

In [20]:
# exog sin ones, porque son constantes

# Dia por dia

In [23]:
def mobilidad(n):
    val = []; X = dummies(n)
    yave = consumo.tail(n); 
    it = np.flip(np.arange(1, n+1))
    for i in it:
        X_train, X_test, y_train, y_test = train_test_split(X, consumo, test_size = i, shuffle = False)
#         y_train, y_test = train_test_split(consumo, test_size = i, shuffle = False)
        model = ARIMA(y_train.values, order=(7,1,0))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast(); predo = output[0]
        val.extend(predo)
    yave["predict"] = val; yave["resta"] = yave.MWh - yave.predict
    yave["error"] = np.abs((yave.MWh - yave.predict)/yave.MWh)*100
    return yave

In [27]:
a = mobilidad(20)
a[["MWh", "predict"]].iplot(title = a.error.mean())

# varios a la vez

In [64]:
def aro(n):
    X = dummies(n)
    X_train, X_test, y_train, y_test = train_test_split(X, consumo, test_size = n, shuffle = False)
    model = ARIMA(y_train.values, exog = X_train,  order=(7,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast(steps = n, exog = X_test); predo = output[0]
    y_test["predict"] = predo
    y_test["resta"] = y_test.MWh - y_test.predict
    y_test["error"] = np.abs((y_test.MWh - y_test.predict)/y_test.MWh)*100
    return y_test

In [65]:
%time ab = aro(7)
ab[["MWh", "predict"]].iplot(title = ab.error.mean())

Wall time: 47.3 s


In [68]:
ab.error

fecha
2020-03-24    0.909146
2020-03-25    2.403920
2020-03-26    2.860092
2020-03-27    2.935731
2020-03-28    2.859508
2020-03-29    2.942188
2020-03-30    1.772369
Name: error, dtype: float64